Analyze() function will solve the equation with FNO uptil FinalSolve time. It will also generate the figures to visualize and give L2 and Inf error between the true and original. As in Input, it takes dictionary of parameters

In [ ]:
from timeit import default_timer

def Analyze(Param):
    %run ./FNO.py
    %run ./Data_Generator/KFRDG.ipynb
    
    %run ./SaveFig.ipynb
    Alpha = Param["Alpha"]
    InitialSolve = Param["InitialSolve"]
    Ndt = Param["Ndt"]
    TrainingSamples = Param["TrainingSamples"]
    FinalSolve = Param["FinalSolve"]
    S = Param["S"]
    Space = Param["Space"]

    Initial = Param["Initial"]
    Cutoff = Param["Cutoff"]
    
    FiguresOnly = Param["FiguresOnly"]
    
    Path_U_IS = 'Data_Generator/' + f'u_original_{Alpha}_{InitialSolve}_{S}.npy' 
    
    U_InitialSolve = torch.tensor(np.load(Path_U_IS, allow_pickle=True))
    
    model = torch.load(f'KFR_FNO_skiptype{Ndt}_alpha{Alpha}_trainingsamples{TrainingSamples}')
    
    
    path_Uth = 'Data_Generator/' + f'u_original_{Alpha}_{FinalSolve}_{S}.npy'
    path_time = 'Data_Generator/' + f'time_{Alpha}_{FinalSolve}_{S}.npy'
    if os.path.exists(path_Uth) and os.path.exists(path_time):
        U_Gt = np.load(path_Uth, allow_pickle=True)
        Time = np.load(path_time, allow_pickle=True)[-1]
    else:
        U_Theoretical = DataGenerator_Alpha(Alpha, TrainingSamples, Ndt, S, S+7, Space, FinalSolve)
        U_Gt = U_Theoretical['u_original']
        Time = U_Theoretical['time'][-1]
    
    L = 6
    N = S
    dx = L / N  # grid size

    x = np.empty(N + 7)
    x[0:3] = [(-L + dx * i) for i in [-3, -2, -1]]  # left ghost points
    x[3:N + 4] = [(-L + dx * i) for i in range(N + 1)]  # internal grid points on [-L,0]
    x[N + 4:N + 7] = np.arange(1, 4) * dx  # grid points ahead of the shock

    Length=np.array([x])[-1]
    
    EndForTime = (Time[::Ndt].shape[-1]-U_InitialSolve[::Ndt,-1].shape[-1])-1
    
    #####
    #Main Loop
    #####
    if not FiguresOnly:
        time = []
        w = model(U_InitialSolve[-1,:].float().reshape(1,S+7,1).cuda())
        time.append(w.cpu().detach().numpy())
        start_timer = default_timer()
        for i in range(0,EndForTime):
            with torch.no_grad():
                w = model(w)
            time.append(w.cpu().detach().numpy())
            
        end_timer = default_timer()
        print("Time for Evaluation: ", end_timer - start_timer)
        print(EndForTime)

        timenp = np.array(time)

        time_shock = timenp[:,-1,:,-1]
        time_pde = timenp[:,-1,:,:]
        
        np.save('Data_Generator/' + f'FNO_shockarray_{Alpha}_{Ndt}',time_shock)
        np.save('Data_Generator/' + f'FNO_PDE_{Alpha}_{Ndt}',time_pde)
        Inf, L2 = FigureSave(Time,Length,U_Gt,U_InitialSolve,time_pde,time_shock,Ndt,Initial,Cutoff,Alpha,TrainingSamples, S)
        
    else:
        path_SA = 'Data_Generator/' + f'FNO_shockarray_{Alpha}_{Ndt}.npy'
        path_FNOPDE = 'Data_Generator/' + f'FNO_PDE_{Alpha}_{Ndt}.npy'
        time_shock = np.load(path_SA, allow_pickle=True)
        time_pde = np.load(path_FNOPDE, allow_pickle=True)
        Inf, L2 = FigureSave(Time,Length,U_Gt,U_InitialSolve,time_pde,time_shock,Ndt,Initial,Cutoff,Alpha,TrainingSamples, S)
    
    return Inf, L2